# CharityWorks Front-End Implementation

This notebook implements the front-end use cases for the CharityWorks project. It covers various functionalities including user authentication, profile management, donation management, campaign and event management, feedback and communication, content display, and other miscellaneous features.

## Setup and Required Libraries

First, let's import the necessary libraries for our front-end implementation.

In [ ]:
# Import necessary libraries
import React from 'react'
import { useState, useEffect } from 'react'
import axios from 'axios'
import { BrowserRouter as Router, Route, Switch, Link } from 'react-router-dom'
import { Container, Form, Button, Card, Alert, Navbar, Nav } from 'react-bootstrap'
import 'bootstrap/dist/css/bootstrap.min.css'

# This is a Jupyter notebook where we'll plan and prototype our React components
# The actual implementation would be in separate .js/.jsx files

# 1. User Authentication

This section covers the implementation of authentication-related pages:
- Login
- Registration
- Logout

In [ ]:
// Login Component
const Login = () => {
  const [email, setEmail] = useState('')
  const [password, setPassword] = useState('')
  const [error, setError] = useState('')
  
  const handleLogin = async (e) => {
    e.preventDefault()
    try {
      const response = await axios.post('/api/auth/login', { email, password })
      localStorage.setItem('token', response.data.token)
      localStorage.setItem('user', JSON.stringify(response.data.user))
      window.location.href = '/dashboard'
    } catch (err) {
      setError('Invalid credentials. Please try again.')
    }
  }
  
  return (
    <Container className="mt-5">
      <h2>Login</h2>
      {error && <Alert variant="danger">{error}</Alert>}
      <Form onSubmit={handleLogin}>
        <Form.Group>
          <Form.Label>Email</Form.Label>
          <Form.Control 
            type="email" 
            value={email} 
            onChange={(e) => setEmail(e.target.value)} 
            required 
          />
        </Form.Group>
        <Form.Group>
          <Form.Label>Password</Form.Label>
          <Form.Control 
            type="password" 
            value={password} 
            onChange={(e) => setPassword(e.target.value)} 
            required 
          />
        </Form.Group>
        <Button type="submit" variant="primary" className="mt-3">Login</Button>
      </Form>
      <div className="mt-3">
        <Link to="/register">Don't have an account? Register here</Link>
      </div>
    </Container>
  )
}

In [ ]:
// Register Component
const Register = () => {
  const [formData, setFormData] = useState({
    name: '',
    email: '',
    password: '',
    confirmPassword: ''
  })
  const [error, setError] = useState('')
  const [success, setSuccess] = useState(false)
  
  const { name, email, password, confirmPassword } = formData
  
  const handleChange = (e) => {
    setFormData({ ...formData, [e.target.name]: e.target.value })
  }
  
  const handleRegister = async (e) => {
    e.preventDefault()
    
    if (password !== confirmPassword) {
      return setError('Passwords do not match')
    }
    
    try {
      await axios.post('/api/auth/register', { name, email, password })
      setSuccess(true)
      setError('')
      // Reset form
      setFormData({
        name: '',
        email: '',
        password: '',
        confirmPassword: ''
      })
    } catch (err) {
      setError(err.response.data.message || 'Registration failed')
    }
  }
  
  return (
    <Container className="mt-5">
      <h2>Register</h2>
      {error && <Alert variant="danger">{error}</Alert>}
      {success && <Alert variant="success">Registration successful! You can now login.</Alert>}
      <Form onSubmit={handleRegister}>
        <Form.Group>
          <Form.Label>Name</Form.Label>
          <Form.Control 
            type="text" 
            name="name"
            value={name} 
            onChange={handleChange} 
            required 
          />
        </Form.Group>
        <Form.Group>
          <Form.Label>Email</Form.Label>
          <Form.Control 
            type="email" 
            name="email"
            value={email} 
            onChange={handleChange} 
            required 
          />
        </Form.Group>
        <Form.Group>
          <Form.Label>Password</Form.Label>
          <Form.Control 
            type="password" 
            name="password"
            value={password} 
            onChange={handleChange} 
            required 
          />
        </Form.Group>
        <Form.Group>
          <Form.Label>Confirm Password</Form.Label>
          <Form.Control 
            type="password" 
            name="confirmPassword"
            value={confirmPassword} 
            onChange={handleChange} 
            required 
          />
        </Form.Group>
        <Button type="submit" variant="primary" className="mt-3">Register</Button>
      </Form>
      <div className="mt-3">
        <Link to="/login">Already have an account? Login here</Link>
      </div>
    </Container>
  )
}

In [ ]:
// Logout functionality
const logoutUser = () => {
  // Clear local storage
  localStorage.removeItem('token')
  localStorage.removeItem('user')
  
  // Redirect to home page
  window.location.href = '/'
}

// Logout Button Component
const LogoutButton = () => {
  return (
    <Button variant="outline-danger" onClick={logoutUser}>
      Logout
    </Button>
  )
}

# 2. User Profile Management

This section covers the implementation of profile-related pages:
- Update user profile
- Edit donor account
- Change language preference

In [ ]:
// Profile Update Component
const ProfileUpdate = () => {
  const [profile, setProfile] = useState({
    name: '',
    email: '',
    phone: '',
    address: '',
    profilePicture: null
  })
  const [loading, setLoading] = useState(true)
  const [error, setError] = useState('')
  const [success, setSuccess] = useState(false)
  
  // Fetch user profile on component mount
  useEffect(() => {
    const fetchProfile = async () => {
      try {
        const token = localStorage.getItem('token')
        const response = await axios.get('/api/users/profile', {
          headers: { Authorization: `Bearer ${token}` }
        })
        setProfile(response.data)
        setLoading(false)
      } catch (err) {
        setError('Error loading profile')
        setLoading(false)
      }
    }
    
    fetchProfile()
  }, [])
  
  const handleChange = (e) => {
    setProfile({ ...profile, [e.target.name]: e.target.value })
  }
  
  const handleFileChange = (e) => {
    setProfile({ ...profile, profilePicture: e.target.files[0] })
  }
  
  const updateProfile = async (e) => {
    e.preventDefault()
    
    try {
      const token = localStorage.getItem('token')
      const formData = new FormData()
      
      for (const key in profile) {
        if (profile[key]) {
          formData.append(key, profile[key])
        }
      }
      
      await axios.put('/api/users/profile', formData, {
        headers: { 
          Authorization: `Bearer ${token}`,
          'Content-Type': 'multipart/form-data'
        }
      })
      
      setSuccess(true)
      setTimeout(() => setSuccess(false), 3000)
    } catch (err) {
      setError('Failed to update profile')
    }
  }
  
  if (loading) return <div>Loading profile...</div>
  
  return (
    <Container className="mt-5">
      <h2>Update Profile</h2>
      {error && <Alert variant="danger">{error}</Alert>}
      {success && <Alert variant="success">Profile updated successfully!</Alert>}
      
      <Form onSubmit={updateProfile}>
        <Form.Group>
          <Form.Label>Name</Form.Label>
          <Form.Control
            type="text"
            name="name"
            value={profile.name}
            onChange={handleChange}
          />
        </Form.Group>
        
        <Form.Group>
          <Form.Label>Email</Form.Label>
          <Form.Control
            type="email"
            name="email"
            value={profile.email}
            onChange={handleChange}
          />
        </Form.Group>
        
        <Form.Group>
          <Form.Label>Phone</Form.Label>
          <Form.Control
            type="text"
            name="phone"
            value={profile.phone || ''}
            onChange={handleChange}
          />
        </Form.Group>
        
        <Form.Group>
          <Form.Label>Address</Form.Label>
          <Form.Control
            as="textarea"
            name="address"
            value={profile.address || ''}
            onChange={handleChange}
          />
        </Form.Group>
        
        <Form.Group>
          <Form.Label>Profile Picture</Form.Label>
          <Form.Control
            type="file"
            accept="image/*"
            onChange={handleFileChange}
          />
        </Form.Group>
        
        <Button type="submit" variant="primary" className="mt-3">
          Update Profile
        </Button>
      </Form>
    </Container>
  )
}

In [ ]:
// Donor Account Edit Component
const DonorAccountEdit = () => {
  const [donorInfo, setDonorInfo] = useState({
    preferredCauses: [],
    recurringDonation: false,
    donationFrequency: 'monthly',
    preferredPaymentMethod: 'credit_card',
    taxReceipts: true
  })
  const [loading, setLoading] = useState(true)
  const [error, setError] = useState('')
  const [success, setSuccess] = useState(false)
  
  // Available causes
  const causes = [
    'Education', 'Healthcare', 'Poverty Relief', 
    'Emergency Aid', 'Orphans', 'Food Security'
  ]
  
  // Fetch donor info on component mount
  useEffect(() => {
    const fetchDonorInfo = async () => {
      try {
        const token = localStorage.getItem('token')
        const response = await axios.get('/api/donors/preferences', {
          headers: { Authorization: `Bearer ${token}` }
        })
        setDonorInfo(response.data)
        setLoading(false)
      } catch (err) {
        setError('Error loading donor information')
        setLoading(false)
      }
    }
    
    fetchDonorInfo()
  }, [])
  
  const handleCheckboxChange = (e) => {
    const { value, checked } = e.target
    
    if (checked) {
      setDonorInfo({
        ...donorInfo,
        preferredCauses: [...donorInfo.preferredCauses, value]
      })
    } else {
      setDonorInfo({
        ...donorInfo,
        preferredCauses: donorInfo.preferredCauses.filter(cause => cause !== value)
      })
    }
  }
  
  const handleChange = (e) => {
    const { name, value, type, checked } = e.target
    setDonorInfo({
      ...donorInfo,
      [name]: type === 'checkbox' ? checked : value
    })
  }
  
  const updateDonorInfo = async (e) => {
    e.preventDefault()
    
    try {
      const token = localStorage.getItem('token')
      await axios.put('/api/donors/preferences', donorInfo, {
        headers: { Authorization: `Bearer ${token}` }
      })
      
      setSuccess(true)
      setTimeout(() => setSuccess(false), 3000)
    } catch (err) {
      setError('Failed to update donor information')
    }
  }
  
  if (loading) return <div>Loading donor information...</div>
  
  return (
    <Container className="mt-5">
      <h2>Edit Donor Preferences</h2>
      {error && <Alert variant="danger">{error}</Alert>}
      {success && <Alert variant="success">Donor information updated successfully!</Alert>}
      
      <Form onSubmit={updateDonorInfo}>
        <Form.Group className="mb-4">
          <Form.Label>Preferred Causes</Form.Label>
          {causes.map(cause => (
            <Form.Check
              key={cause}
              type="checkbox"
              label={cause}
              value={cause}
              checked={donorInfo.preferredCauses.includes(cause)}
              onChange={handleCheckboxChange}
            />
          ))}
        </Form.Group>
        
        <Form.Group className="mb-4">
          <Form.Check
            type="checkbox"
            label="Set up recurring donations"
            name="recurringDonation"
            checked={donorInfo.recurringDonation}
            onChange={handleChange}
          />
        </Form.Group>
        
        {donorInfo.recurringDonation && (
          <Form.Group className="mb-4">
            <Form.Label>Donation Frequency</Form.Label>
            <Form.Select 
              name="donationFrequency" 
              value={donorInfo.donationFrequency}
              onChange={handleChange}
            >
              <option value="weekly">Weekly</option>
              <option value="monthly">Monthly</option>
              <option value="quarterly">Quarterly</option>
              <option value="annually">Annually</option>
            </Form.Select>
          </Form.Group>
        )}
        
        <Form.Group className="mb-4">
          <Form.Label>Preferred Payment Method</Form.Label>
          <Form.Select 
            name="preferredPaymentMethod" 
            value={donorInfo.preferredPaymentMethod}
            onChange={handleChange}
          >
            <option value="credit_card">Credit Card</option>
            <option value="bank_transfer">Bank Transfer</option>
            <option value="paypal">PayPal</option>
          </Form.Select>
        </Form.Group>
        
        <Form.Group className="mb-4">
          <Form.Check
            type="checkbox"
            label="Receive tax receipts for donations"
            name="taxReceipts"
            checked={donorInfo.taxReceipts}
            onChange={handleChange}
          />
        </Form.Group>
        
        <Button type="submit" variant="primary">
          Save Preferences
        </Button>
      </Form>
    </Container>
  )
}

In [ ]:
// Language Preference Component
const LanguageSelector = () => {
  const [currentLang, setCurrentLang] = useState(() => {
    return localStorage.getItem('language') || 'en'
  })
  
  const languages = [
    { code: 'en', name: 'English' },
    { code: 'ar', name: 'Arabic' },
    { code: 'fr', name: 'French' },
    { code: 'es', name: 'Spanish' }
  ]
  
  const changeLanguage = (langCode) => {
    localStorage.setItem('language', langCode)
    setCurrentLang(langCode)
    // In a real app, you would also update the UI language
    // using an i18n library like i18next or react-intl
    document.documentElement.lang = langCode
    
    // If using i18next:
    // i18n.changeLanguage(langCode)
  }
  
  return (
    <div className="language-selector">
      <div className="d-flex align-items-center">
        <span className="me-2">Language:</span>
        <Form.Select 
          size="sm" 
          value={currentLang} 
          onChange={(e) => changeLanguage(e.target.value)}
          style={{ width: 'auto' }}
        >
          {languages.map((lang) => (
            <option key={lang.code} value={lang.code}>
              {lang.name}
            </option>
          ))}
        </Form.Select>
      </div>
    </div>
  )
}

# 3. Donation Management

This section covers the implementation of donation-related pages:
- Making donations
- Canceling donations
- Tracking donations
- Choosing donation types
- Calculating Zakah
- Selecting payment methods
- Viewing donation history
- Generating donation reports

In [ ]:
// Make Donation Component
const MakeDonation = () => {
  const [donationForm, setDonationForm] = useState({
    amount: '',
    campaign: '',
    donationType: 'one-time',
    anonymous: false,
    message: '',
    paymentMethod: 'credit_card'
  })
  const [campaigns, setCampaigns] = useState([])
  const [loading, setLoading] = useState(true)
  const [error, setError] = useState('')
  const [success, setSuccess] = useState(false)
  
  // Fetch available campaigns
  useEffect(() => {
    const fetchCampaigns = async () => {
      try {
        const response = await axios.get('/api/campaigns')
        setCampaigns(response.data)
        setLoading(false)
      } catch (err) {
        setError('Error loading campaigns')
        setLoading(false)
      }
    }
    
    fetchCampaigns()
  }, [])
  
  const handleChange = (e) => {
    const { name, value, type, checked } = e.target
    setDonationForm({
      ...donationForm,
      [name]: type === 'checkbox' ? checked : value
    })
  }
  
  const handleSubmit = async (e) => {
    e.preventDefault()
    
    try {
      const token = localStorage.getItem('token')
      await axios.post('/api/donations', donationForm, {
        headers: { Authorization: `Bearer ${token}` }
      })
      
      setSuccess(true)
      // Reset form
      setDonationForm({
        amount: '',
        campaign: '',
        donationType: 'one-time',
        anonymous: false,
        message: '',
        paymentMethod: 'credit_card'
      })
      
      setTimeout(() => setSuccess(false), 3000)
    } catch (err) {
      setError('Failed to process donation. Please try again.')
    }
  }
  
  if (loading) return <div>Loading campaigns...</div>
  
  return (
    <Container className="mt-5">
      <h2>Make a Donation</h2>
      {error && <Alert variant="danger">{error}</Alert>}
      {success && <Alert variant="success">Thank you for your donation!</Alert>}
      
      <Form onSubmit={handleSubmit}>
        <Form.Group className="mb-3">
          <Form.Label>Amount ($)</Form.Label>
          <Form.Control
            type="number"
            name="amount"
            value={donationForm.amount}
            onChange={handleChange}
            min="1"
            required
          />
        </Form.Group>
        
        <Form.Group className="mb-3">
          <Form.Label>Campaign</Form.Label>
          <Form.Select
            name="campaign"
            value={donationForm.campaign}
            onChange={handleChange}
            required
          >
            <option value="">Select a campaign</option>
            {campaigns.map(campaign => (
              <option key={campaign.id} value={campaign.id}>
                {campaign.title}
              </option>
            ))}
          </Form.Select>
        </Form.Group>
        
        <Form.Group className="mb-3">
          <Form.Label>Donation Type</Form.Label>
          <Form.Select
            name="donationType"
            value={donationForm.donationType}
            onChange={handleChange}
          >
            <option value="one-time">One-time donation</option>
            <option value="monthly">Monthly recurring</option>
            <option value="quarterly">Quarterly recurring</option>
            <option value="annually">Annual recurring</option>
          </Form.Select>
        </Form.Group>
        
        <Form.Group className="mb-3">
          <Form.Check
            type="checkbox"
            label="Make this donation anonymous"
            name="anonymous"
            checked={donationForm.anonymous}
            onChange={handleChange}
          />
        </Form.Group>
        
        <Form.Group className="mb-3">
          <Form.Label>Message (Optional)</Form.Label>
          <Form.Control
            as="textarea"
            name="message"
            value={donationForm.message}
            onChange={handleChange}
            rows={3}
          />
        </Form.Group>
        
        <Form.Group className="mb-3">
          <Form.Label>Payment Method</Form.Label>
          <Form.Select
            name="paymentMethod"
            value={donationForm.paymentMethod}
            onChange={handleChange}
          >
            <option value="credit_card">Credit Card</option>
            <option value="paypal">PayPal</option>
            <option value="bank_transfer">Bank Transfer</option>
          </Form.Select>
        </Form.Group>
        
        <Button type="submit" variant="primary">
          Donate Now
        </Button>
      </Form>
    </Container>
  )
}

In [ ]:
// Cancel Donation Component
const CancelDonation = () => {
  const [recurringDonations, setRecurringDonations] = useState([])
  const [loading, setLoading] = useState(true)
  const [error, setError] = useState('')
  const [success, setSuccess] = useState(false)
  
  // Fetch user's recurring donations
  useEffect(() => {
    const fetchRecurringDonations = async () => {
      try {
        const token = localStorage.getItem('token')
        const response = await axios.get('/api/donations/recurring', {
          headers: { Authorization: `Bearer ${token}` }
        })
        setRecurringDonations(response.data)
        setLoading(false)
      } catch (err) {
        setError('Error loading recurring donations')
        setLoading(false)
      }
    }
    
    fetchRecurringDonations()
  }, [])
  
  const handleCancelDonation = async (donationId) => {
    try {
      const token = localStorage.getItem('token')
      await axios.post(`/api/donations/${donationId}/cancel`, {}, {
        headers: { Authorization: `Bearer ${token}` }
      })
      
      // Remove the cancelled donation from the list
      setRecurringDonations(
        recurringDonations.filter(donation => donation.id !== donationId)
      )
      
      setSuccess(true)
      setTimeout(() => setSuccess(false), 3000)
    } catch (err) {
      setError('Failed to cancel donation. Please try again.')
    }
  }
  
  if (loading) return <div>Loading recurring donations...</div>
  
  return (
    <Container className="mt-5">
      <h2>Manage Recurring Donations</h2>
      {error && <Alert variant="danger">{error}</Alert>}
      {success && <Alert variant="success">Donation cancelled successfully!</Alert>}
      
      {recurringDonations.length === 0 ? (
        <Alert variant="info">You don't have any active recurring donations.</Alert>
      ) : (
        <div>
          {recurringDonations.map(donation => (
            <Card key={donation.id} className="mb-3">
              <Card.Body>
                <Card.Title>${donation.amount} - {donation.campaign.title}</Card.Title>
                <Card.Subtitle className="mb-2 text-muted">
                  {donation.donationType} | Next payment: {new Date(donation.nextPaymentDate).toLocaleDateString()}
                </Card.Subtitle>
                <Button 
                  variant="danger"
                  onClick={() => handleCancelDonation(donation.id)}
                >
                  Cancel this Recurring Donation
                </Button>
              </Card.Body>
            </Card>
          ))}
        </div>
      )}
    </Container>
  )
}

In [ ]:
// Track Donations Component
const TrackDonations = () => {
  const [donations, setDonations] = useState([])
  const [loading, setLoading] = useState(true)
  const [error, setError] = useState('')
  const [filters, setFilters] = useState({
    startDate: '',
    endDate: '',
    campaign: '',
    minAmount: '',
    maxAmount: ''
  })
  
  // Fetch user's donations
  useEffect(() => {
    const fetchDonations = async () => {
      try {
        const token = localStorage.getItem('token')
        const response = await axios.get('/api/donations/history', {
          headers: { Authorization: `Bearer ${token}` }
        })
        setDonations(response.data)
        setLoading(false)
      } catch (err) {
        setError('Error loading donation history')
        setLoading(false)
      }
    }
    
    fetchDonations()
  }, [])
  
  const handleFilterChange = (e) => {
    const { name, value } = e.target
    setFilters({
      ...filters,
      [name]: value
    })
  }
  
  const applyFilters = () => {
    // This would typically make a backend request with filters
    // For now, we'll simulate filtering the existing data
    const fetchFilteredDonations = async () => {
      try {
        setLoading(true)
        const token = localStorage.getItem('token')
        
        // Build query parameters
        const params = new URLSearchParams()
        if (filters.startDate) params.append('startDate', filters.startDate)
        if (filters.endDate) params.append('endDate', filters.endDate)
        if (filters.campaign) params.append('campaign', filters.campaign)
        if (filters.minAmount) params.append('minAmount', filters.minAmount)
        if (filters.maxAmount) params.append('maxAmount', filters.maxAmount)
        
        const response = await axios.get(`/api/donations/history?${params.toString()}`, {
          headers: { Authorization: `Bearer ${token}` }
        })
        
        setDonations(response.data)
        setLoading(false)
      } catch (err) {
        setError('Error filtering donations')
        setLoading(false)
      }
    }
    
    fetchFilteredDonations()
  }
  
  if (loading) return <div>Loading donation history...</div>
  
  return (
    <Container className="mt-5">
      <h2>Track Your Donations</h2>
      {error && <Alert variant="danger">{error}</Alert>}
      
      <Card className="mb-4">
        <Card.Body>
          <Card.Title>Filter Donations</Card.Title>
          <Form>
            <Row>
              <Col md={6}>
                <Form.Group className="mb-3">
                  <Form.Label>Start Date</Form.Label>
                  <Form.Control
                    type="date"
                    name="startDate"
                    value={filters.startDate}
                    onChange={handleFilterChange}
                  />
                </Form.Group>
              </Col>
              <Col md={6}>
                <Form.Group className="mb-3">
                  <Form.Label>End Date</Form.Label>
                  <Form.Control
                    type="date"
                    name="endDate"
                    value={filters.endDate}
                    onChange={handleFilterChange}
                  />
                </Form.Group>
              </Col>
            </Row>
            
            <Row>
              <Col md={4}>
                <Form.Group className="mb-3">
                  <Form.Label>Campaign</Form.Label>
                  <Form.Control
                    type="text"
                    name="campaign"
                    value={filters.campaign}
                    onChange={handleFilterChange}
                    placeholder="Campaign name"
                  />
                </Form.Group>
              </Col>
              <Col md={4}>
                <Form.Group className="mb-3">
                  <Form.Label>Min Amount ($)</Form.Label>
                  <Form.Control
                    type="number"
                    name="minAmount"
                    value={filters.minAmount}
                    onChange={handleFilterChange}
                    placeholder="Min amount"
                  />
                </Form.Group>
              </Col>
              <Col md={4}>
                <Form.Group className="mb-3">
                  <Form.Label>Max Amount ($)</Form.Label>
                  <Form.Control
                    type="number"
                    name="maxAmount"
                    value={filters.maxAmount}
                    onChange={handleFilterChange}
                    placeholder="Max amount"
                  />
                </Form.Group>
              </Col>
            </Row>
            
            <Button variant="primary" onClick={applyFilters}>
              Apply Filters
            </Button>
          </Form>
        </Card.Body>
      </Card>
      
      <h3>Donation History</h3>
      {donations.length === 0 ? (
        <Alert variant="info">No donations found matching your criteria.</Alert>
      ) : (
        <Table striped bordered hover>
          <thead>
            <tr>
              <th>Date</th>
              <th>Campaign</th>
              <th>Amount</th>
              <th>Type</th>
              <th>Status</th>
            </tr>
          </thead>
          <tbody>
            {donations.map(donation => (
              <tr key={donation.id}>
                <td>{new Date(donation.date).toLocaleDateString()}</td>
                <td>{donation.campaign.title}</td>
                <td>${donation.amount.toFixed(2)}</td>
                <td>{donation.donationType}</td>
                <td>
                  <Badge bg={donation.status === 'completed' ? 'success' : 'warning'}>
                    {donation.status}
                  </Badge>
                </td>
              </tr>
            ))}
          </tbody>
        </Table>
      )}
    </Container>
  )
}

In [ ]:
// Zakah Calculator Component
const ZakahCalculator = () => {
  const [assets, setAssets] = useState({
    cash: 0,
    gold: 0,
    silver: 0,
    stocks: 0,
    properties: 0,
    businessAssets: 0,
    otherAssets: 0
  })
  
  const [liabilities, setLiabilities] = useState({
    loans: 0,
    debts: 0,
    otherLiabilities: 0
  })
  
  const [zakahResult, setZakahResult] = useState({
    totalAssets: 0,
    totalLiabilities: 0,
    netWorth: 0,
    zakahPayable: 0,
    nisabThreshold: 4500 // Example threshold in USD
  })
  
  // Handle changes to asset fields
  const handleAssetChange = (e) => {
    const { name, value } = e.target
    setAssets({
      ...assets,
      [name]: parseFloat(value) || 0
    })
  }
  
  // Handle changes to liability fields
  const handleLiabilityChange = (e) => {
    const { name, value } = e.target
    setLiabilities({
      ...liabilities,
      [name]: parseFloat(value) || 0
    })
  }
  
  // Calculate Zakah
  const calculateZakah = (e) => {
    e.preventDefault()
    
    const totalAssets = Object.values(assets).reduce((sum, value) => sum + value, 0)
    const totalLiabilities = Object.values(liabilities).reduce((sum, value) => sum + value, 0)
    const netWorth = totalAssets - totalLiabilities
    
    // Zakah is 2.5% of net worth if it's above nisab threshold
    const zakahPayable = netWorth > zakahResult.nisabThreshold ? netWorth * 0.025 : 0
    
    setZakahResult({
      ...zakahResult,
      totalAssets,
      totalLiabilities,
      netWorth,
      zakahPayable
    })
  }
  
  return (
    <Container className="mt-5">
      <h2>Zakah Calculator</h2>
      <p>Calculate your annual Zakah obligation based on your assets and liabilities.</p>
      
      <Form onSubmit={calculateZakah}>
        <Row>
          <Col md={6}>
            <Card className="mb-4">
              <Card.Body>
                <Card.Title>Assets</Card.Title>
                <Form.Group className="mb-3">
                  <Form.Label>Cash on Hand & in Bank</Form.Label>
                  <Form.Control
                    type="number"
                    name="cash"
                    value={assets.cash}
                    onChange={handleAssetChange}
                  />
                </Form.Group>
                
                <Form.Group className="mb-3">
                  <Form.Label>Gold (value in $)</Form.Label>
                  <Form.Control
                    type="number"
                    name="gold"
                    value={assets.gold}
                    onChange={handleAssetChange}
                  />
                </Form.Group>
                
                <Form.Group className="mb-3">
                  <Form.Label>Silver (value in $)</Form.Label>
                  <Form.Control
                    type="number"
                    name="silver"
                    value={assets.silver}
                    onChange={handleAssetChange}
                  />
                </Form.Group>
                
                <Form.Group className="mb-3">
                  <Form.Label>Stocks & Investments (value in $)</Form.Label>
                  <Form.Control
                    type="number"
                    name="stocks"
                    value={assets.stocks}
                    onChange={handleAssetChange}
                  />
                </Form.Group>
                
                <Form.Group className="mb-3">
                  <Form.Label>Investment Properties (value in $)</Form.Label>
                  <Form.Control
                    type="number"
                    name="properties"
                    value={assets.properties}
                    onChange={handleAssetChange}
                  />
                </Form.Group>
                
                <Form.Group className="mb-3">
                  <Form.Label>Business Assets & Inventory (value in $)</Form.Label>
                  <Form.Control
                    type="number"
                    name="businessAssets"
                    value={assets.businessAssets}
                    onChange={handleAssetChange}
                  />
                </Form.Group>
                
                <Form.Group className="mb-3">
                  <Form.Label>Other Assets (value in $)</Form.Label>
                  <Form.Control
                    type="number"
                    name="otherAssets"
                    value={assets.otherAssets}
                    onChange={handleAssetChange}
                  />
                </Form.Group>
              </Card.Body>
            </Card>
          </Col>
          
          <Col md={6}>
            <Card className="mb-4">
              <Card.Body>
                <Card.Title>Liabilities</Card.Title>
                <Form.Group className="mb-3">
                  <Form.Label>Loans Outstanding</Form.Label>
                  <Form.Control
                    type="number"
                    name="loans"
                    value={liabilities.loans}
                    onChange={handleLiabilityChange}
                  />
                </Form.Group>
                
                <Form.Group className="mb-3">
                  <Form.Label>Debts Owed</Form.Label>
                  <Form.Control
                    type="number"
                    name="debts"
                    value={liabilities.debts}
                    onChange={handleLiabilityChange}
                  />
                </Form.Group>
                
                <Form.Group className="mb-3">
                  <Form.Label>Other Liabilities</Form.Label>
                  <Form.Control
                    type="number"
                    name="otherLiabilities"
                    value={liabilities.otherLiabilities}
                    onChange={handleLiabilityChange}
                  />
                </Form.Group>
              </Card.Body>
            </Card>
            
            <Button type="submit" variant="primary" size="lg" className="w-100">
              Calculate Zakah
            </Button>
            
            {zakahResult.totalAssets > 0 && (
              <Card className="mt-4 bg-light">
                <Card.Body>
                  <Card.Title>Zakah Calculation Results</Card.Title>
                  <Table bordered>
                    <tbody>
                      <tr>
                        <td>Total Assets:</td>
                        <td>${zakahResult.totalAssets.toFixed(2)}</td>
                      </tr>
                      <tr>
                        <td>Total Liabilities:</td>
                        <td>${zakahResult.totalLiabilities.toFixed(2)}</td>
                      </tr>
                      <tr>
                        <td>Net Worth:</td>
                        <td>${zakahResult.netWorth.toFixed(2)}</td>
                      </tr>
                      <tr>
                        <td>Nisab Threshold:</td>
                        <td>${zakahResult.nisabThreshold.toFixed(2)}</td>
                      </tr>
                      <tr className="table-primary">
                        <td><strong>Zakah Payable:</strong></td>
                        <td>
                          <strong>
                            ${zakahResult.zakahPayable.toFixed(2)}
                          </strong>
                          {zakahResult.zakahPayable === 0 && (
                            <div className="text-muted small">
                              (Your net worth is below Nisab threshold)
                            </div>
                          )}
                        </td>
                      </tr>
                    </tbody>
                  </Table>
                  
                  {zakahResult.zakahPayable > 0 && (
                    <div className="mt-3">
                      <Button variant="success">
                        Pay Zakah Now (${zakahResult.zakahPayable.toFixed(2)})
                      </Button>
                    </div>
                  )}
                </Card.Body>
              </Card>
            )}
          </Col>
        </Row>
      </Form>
    </Container>
  )
}